# Escalado

In [1]:
import pickle

In [2]:
import numpy as np

import pandas as pd

% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set()

from sklearn.preprocessing import StandardScaler

In [3]:
sample = pd.read_pickle("data/sample_all.pkl.bz2")

## Removemos los features que no queremos


In [5]:
to_drop = [
    'Gskew', # has nan in the rlyrae stars
    "scls_h", "scls_j", "scls_k",  # no nos sirve
    "AndersonDarling", "StetsonJ", "StetsonK"
] + [s for s in sample.columns if s.startswith("Freq2_") or s.startswith("Freq3_") ] # only the first period is important
to_keep = [c for c in sample.columns if c not in to_drop]

sample = sample[to_keep]

Removemos filas que tengan un nan en `period_fit` pero antes nos fijamos que ninguna sea una RR-Lyrae

In [6]:
print sample[sample.Period_fit.isnull()].vs_type.unique()

['']


In [7]:
sample = sample.dropna()

Evaluamos que columnas tienen un valor infinito

In [8]:
for x in sample.columns:
    if sample[x].dtype == object:
        continue
    if np.isinf(sample[x].values).sum():
        print x

Period_fit


Como period_fit es un feature que me interesa, verificamos que cantidad de filas son las afectadas
y si hay alguna con RRLyraes

In [9]:
print sample[np.isinf(sample.Period_fit.values)].shape
print sample[np.isinf(sample.Period_fit.values)].vs_type.unique()


(3, 62)
['']


Son pocas filas y no hay rrlyraes... las borramos

In [10]:
sample = sample[~np.isinf(sample.Period_fit.values)]

ahora almacenamos tod esto limpio para futuros usos

In [12]:
sample.to_pickle("data/nonull/sample.pkl.bz2", compression="bz2")

## Ahora normalizamos

In [13]:
no_features = ["id", "vs_catalog", "vs_type", "ra_k", "dec_k", ]
X_columns = [c for c in sample.columns if c not in no_features]

In [15]:
scaler = StandardScaler()
sample = sample.copy()
sample[X_columns] = scaler.fit_transform(sample[X_columns])
pickle.dump(scaler, open("data/scalers/scaler.pkl", "wb"))
sample.to_pickle("data/scaled/sample.pkl.bz2", compression="bz2")

/home/jbcabral/.conda/envs/carpyncho3/lib/python2.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jbcabral/.conda/envs/carpyncho3/lib/python2.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
